In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 경로지정
dataPath = "/content/drive/MyDrive/LLM/"

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
!pip install jsonlines

In [ ]:
!pip install datasets

In [ ]:
import jsonlines

In [ ]:
import pandas as pd
import json

In [ ]:
from datasets import Dataset

In [ ]:
datasetName = "LLM_dataset_CrayonShinchan.csv"
jsonFileName = "LLM_dataset_CrayonShinchan.jsonl"

In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    df = pd.read_csv(csv_file_path)

    with open(json_file_path, 'w', encoding = 'utf-8') as json_file:
        for index, row in df.iterrows():
            data = {'Inputs': row['inputs'], 'Response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')

csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

csv_to_json(csv_file_path, json_file_path)

In [ ]:
import jsonlines
indataset = []
with jsonlines.open(json_file_path) as f:
  for line in f:
    indataset.append(f'<s>[INST] {line["Inputs"]} [/INST] {line["Response"]} </s>')

print('데이터셋 확인')
print(indataset[:5])

indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

print('데이터셋 info 확인')
print(indataset)

데이터셋 확인
['<s>[INST] 짱구는 어떤 애니메이션의 등장인물입니까? [/INST] 짱구는 못말려입니다. </s>', '<s>[INST] 짱구는 몇 살입니까? [/INST] 5살입니다. </s>', '<s>[INST] 짱구의 가족 구성원은 어떻게 됩니까? [/INST] 아버지, 어머니, 여동생, 강아지가 있습니다. </s>', '<s>[INST] 가족 구성원들의 이름은 무엇입니까? [/INST] 아버지는 신형만, 어머니는 봉미선, 여동생은 신짱아, 강아지는 흰둥이입니다. </s>', '<s>[INST] 짱구는 어디에 소속되어 있나요? [/INST] 떡잎유치원 해바라기반 소속입니다. </s>']


Saving the dataset (0/1 shards):   0%|          | 0/29 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 29
})


In [ ]:
indataset.push_to_hub("eomjiyeon/custom_LLM")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/eomjiyeon/custom_LLM/commit/a874d193f4280ad417858b17e5ae09e89514be49', commit_message='Upload dataset', commit_description='', oid='a874d193f4280ad417858b17e5ae09e89514be49', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
# Model from Hugging Face hub
base_model = "beomi/llama-2-ko-7b"

# New instruction dataset
guanaco_dataset = "eomjiyeon/custom_LLM"

# Fine-tuned model
new_model = "llama-2-7b-chat-custom_LLM"

In [ ]:
dataset = load_dataset(guanaco_dataset, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
print(dataset[10])

{'text': '<s>[INST] 짱구의 거주지는 어디입니까? [/INST] 서울특별시 떡잎마을에 살고 있습니다. </s>'}


In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

model-00013-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
text = "안녕하세요?"
tokens = tokenizer.tokenize(text)
print(tokens)

In [ ]:
peft_params = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = "none",
    task_type = "CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,2.714400
50,1.387000
75,0.896300


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=80, training_loss=1.5925498247146606, metrics={'train_runtime': 180.9779, 'train_samples_per_second': 1.602, 'train_steps_per_second': 0.442, 'total_flos': 498049624719360.0, 'train_loss': 1.5925498247146606, 'epoch': 10.0})

In [ ]:
output_dir = "/content/drive/MyDrive/LLM_model/"

trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
logging.set_verbosity(logging.CRITICAL)
prompt = "짱구는 어떤 애니메이션의 등장인물입니까?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=50)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s>[INST] 짱구는 어떤 애니메이션의 등장인물입니까? [/INST] 짱구는 못말려입니다. 5. 애니메이션에서 가장 좋아하는 캐릭터는? 6. 가장 좋아하는 애니메이션은? 
